In [19]:
import torch
from torch.utils.data import DataLoader, random_split
import torchvision.datasets as datasets 
import torchvision.transforms as transforms
import torch.nn as nn
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
import os
import numpy as np
import pandas as pd
from Model import MLP

from tensorflow.keras.layers import (Dense,Flatten)
from tensorflow.keras.models import Sequential, clone_model

from collections import namedtuple

from tensorflow.keras.utils import to_categorical

from Quantized_network import QuantizedNeuralNetwork, Sequence, _bit_round_parallel

from time import time

In [20]:
def train_network(name, parameters, model,X_test,y_test,X_val,y_val) -> pd.DataFrame:

    num_layers = sum([layer.__class__.__name__ in ('Dense',) for layer in model.layers])
    
    quant_train_size = len(y_val)
    get_data = Sequence(X_val,y_val, batch_size=quant_train_size)
    batch_size = quant_train_size
    my_quant_net = QuantizedNeuralNetwork(
        network=model,
        batch_size=batch_size,
        get_data=get_data,
        bits=parameters.bits,
        alphabet_scalar=parameters.alphabet_scalar,
    )
    tic = time()
    my_quant_net.quantize_network()
    quantization_time = time() - tic

    my_quant_net.quantized_net.compile(
        optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"]
    )
    _, q_accuracy = my_quant_net.quantized_net.evaluate(X_test, y_test, verbose=True)

    MSQ_model = clone_model(model)
   
    MSQ_model.set_weights(model.get_weights())
    
    for layer_idx, layer in enumerate(model.layers):
        if (
            layer.__class__.__name__ in ("Dense")
        ):
            W, b = model.layers[layer_idx].get_weights()
            rad = parameters.alphabet_scalar * np.median(np.abs(W.flatten()))
            layer_alphabet = rad*my_quant_net.alphabet
            Q = np.array([_bit_round_parallel(w, layer_alphabet) for w in W.flatten()]).reshape(
                W.shape
            )
            MSQ_model.layers[layer_idx].set_weights([Q, b])

    MSQ_model.compile(
        optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"]
    )
    _, MSQ_accuracy = MSQ_model.evaluate(X_test, y_test, verbose=True)

    trial_metrics = pd.DataFrame(
        {
            "q_train_size": batch_size,
            "bits": parameters.bits,
            "alphabet_scalar": parameters.alphabet_scalar,
            "sd_test_acc": q_accuracy,
            "msq_test_acc": MSQ_accuracy,
            "quantization_time": quantization_time
        },
        index=[name],
    )

    return trial_metrics

# MNIST

In [22]:
_ = torch.manual_seed(0)

In [23]:
transform = transforms.Compose([transforms.ToTensor()])

mnist_train_valset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

dataset_size = len(mnist_train_valset)
train_size = int(0.9 * dataset_size)
val_size = dataset_size - train_size
train_dataset, val_dataset = random_split(mnist_train_valset, [train_size, val_size])

batch_size = 10

mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
device = "cpu"

## Importing pretrained model

In [24]:
net = MLP(input_size=28*28, output_size=10).to(device)

MODEL_FILENAME = './Saved_models/Mnist.pt'

if Path(MODEL_FILENAME).exists():
    net.load_state_dict(torch.load(MODEL_FILENAME))
    print('Loaded model from disk')
else:
    print('There is no such model')

Loaded model from disk


In [25]:
model = Sequential()
model.add(Flatten(input_shape=(28, 28),))
model.add(Dense(100,use_bias=True, activation="relu"))
model.add(Dense(100,use_bias=True, activation="relu"))   
model.add(Dense(10))
   
torch_weights = net.state_dict()
keras_weights = [w.numpy() for w in torch_weights.values()]
for i in [0, 2, 4]:
    keras_weights[i] = keras_weights[i].T

model.set_weights(keras_weights)

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_6 (Dense)             (None, 100)               78500     
                                                                 
 dense_7 (Dense)             (None, 100)               10100     
                                                                 
 dense_8 (Dense)             (None, 10)                1010      
                                                                 
Total params: 89610 (350.04 KB)
Trainable params: 89610 (350.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Quantization

In [26]:
y_test = mnist_testset.targets.reshape(-1,1).numpy()
X_test = mnist_testset.data.numpy()

y_val = val_dataset.dataset.targets[val_dataset.indices].reshape(-1,1).numpy()
X_val = val_dataset.dataset.data[val_dataset.indices].numpy()

num_classes = np.unique(y_test).shape[0]
y_val = to_categorical(y_val, num_classes)
y_test = to_categorical(y_test, num_classes)

parameter = [np.log2(2**8),2]
ParamConfig = namedtuple("ParamConfig", "bits, alphabet_scalar")

trial_metrics = train_network('MNIST', ParamConfig(*parameter), model, X_test,y_test,X_val,y_val)

Quantizing layer 1 (in parallel) of 4...
	Feeding input data through hidden layers...
	done. 0.826966 seconds.
	Quantizing neurons (in parallel)...
		Neuron 3 of 100 quantized successfully.
		Neuron 2 of 100 quantized successfully.
		Neuron 1 of 100 quantized successfully.
		Neuron 0 of 100 quantized successfully.
		Neuron 4 of 100 quantized successfully.
		Neuron 5 of 100 quantized successfully.
		Neuron 6 of 100 quantized successfully.
		Neuron 7 of 100 quantized successfully.
		Neuron 8 of 100 quantized successfully.
		Neuron 10 of 100 quantized successfully.
		Neuron 11 of 100 quantized successfully.
		Neuron 9 of 100 quantized successfully.
		Neuron 12 of 100 quantized successfully.
		Neuron 15 of 100 quantized successfully.
		Neuron 13 of 100 quantized successfully.
		Neuron 14 of 100 quantized successfully.
		Neuron 16 of 100 quantized successfully.
		Neuron 18 of 100 quantized successfully.
		Neuron 17 of 100 quantized successfully.
		Neuron 19 of 100 quantized successfully.
		

		Neuron 83 of 100 quantized successfully.
		Neuron 84 of 100 quantized successfully.
		Neuron 85 of 100 quantized successfully.
		Neuron 86 of 100 quantized successfully.
		Neuron 87 of 100 quantized successfully.
		Neuron 88 of 100 quantized successfully.
		Neuron 89 of 100 quantized successfully.
		Neuron 90 of 100 quantized successfully.
		Neuron 91 of 100 quantized successfully.
		Neuron 92 of 100 quantized successfully.
		Neuron 93 of 100 quantized successfully.
		Neuron 94 of 100 quantized successfully.
		Neuron 96 of 100 quantized successfully.
		Neuron 95 of 100 quantized successfully.
		Neuron 97 of 100 quantized successfully.
		Neuron 98 of 100 quantized successfully.
		Neuron 99 of 100 quantized successfully.
	done. 22.14 seconds.
Layer 2 of 4 quantized successfully in 22.53 seconds.
Quantizing layer 3 (in parallel) of 4...
	Feeding input data through hidden layers...
	done. 0.368282 seconds.
	Quantizing neurons (in parallel)...
		Neuron 1 of 10 quantized successfully.
		Ne

In [27]:
trial_metrics

,q_train_size,bits,alphabet_scalar,sd_test_acc,msq_test_acc,quantization_time
MNIST,6000,8.0,2,0.5689,0.6988,105.39086


# PLANE

In [28]:
df_train = pd.read_csv('./data/Plane_dataset/train.csv', index_col=0)
df_train['Arrival Delay in Minutes'] = df_train['Arrival Delay in Minutes'].fillna(0)

In [29]:
df_test = pd.read_csv('./data/Plane_dataset/test.csv', index_col=0)
df_test = df_test.drop(columns=['id']).dropna()

In [30]:
from sklearn.model_selection import train_test_split

X_train_val = df_train.iloc[:, :-1].copy()
y_train_val = df_train.iloc[:, -1].copy()

X_train_val = X_train_val.drop(columns=['id'])

y_train_val = pd.get_dummies(y_train_val, drop_first=True)

X_train, X_val, y_train, y_val_ = train_test_split(X_train_val, y_train_val, test_size=0.1, random_state=102)

X_test = df_test.iloc[:, :-1].copy()
y_test = df_test.iloc[:, -1].copy()

y_test_ = pd.get_dummies(y_test, drop_first=True)

In [31]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

num_cols = ['Age', 'Flight Distance', 'Departure Delay in Minutes', 'Arrival Delay in Minutes']
cat_cols = X_train_val.drop(columns=num_cols).columns.tolist()

X_scaler = StandardScaler()
one_hot = OneHotEncoder(drop='if_binary', sparse_output=False, handle_unknown='ignore')

ct = ColumnTransformer([
        ('one_hot', one_hot, cat_cols),
        ('scaler', X_scaler, num_cols)
])

X_train_transformed_ = ct.fit_transform(X_train)
X_val_transformed_ = ct.transform(X_val)
X_test_transformed_ = ct.transform(X_test)

## Importing pretrained model

In [32]:
net = MLP(input_size=93, output_size=2).to(device)

In [33]:
MODEL_FILENAME = './Saved_models/Plane.pt'

if Path(MODEL_FILENAME).exists():
    net.load_state_dict(torch.load(MODEL_FILENAME))
    print('Loaded model from disk')
else:
    print('There is no such model')

Loaded model from disk


In [34]:
model = Sequential()
model.add(Dense(100,input_shape=(93,), use_bias=True, activation="relu"))
model.add(Dense(100,use_bias=True, activation="relu"))   
model.add(Dense(2))
   
torch_weights = net.state_dict()
keras_weights = [w.numpy() for w in torch_weights.values()]
for i in [0, 2, 4]:
    keras_weights[i] = keras_weights[i].T

model.set_weights(keras_weights)

model.compile(optimizer="adam", loss='binary_crossentropy', metrics=["accuracy"])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 100)               9400      
                                                                 
 dense_10 (Dense)            (None, 100)               10100     
                                                                 
 dense_11 (Dense)            (None, 2)                 202       
                                                                 
Total params: 19702 (76.96 KB)
Trainable params: 19702 (76.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Quantization

In [35]:
X_test = X_test_transformed_
y_test = y_test_.values

trial_metrics = train_network('plane', ParamConfig(*parameter), model,X_test,y_test,X_val_transformed_, y_val_.values)

Quantizing layer 0 (in parallel) of 3...
	Feeding input data through hidden layers...
	done. 0.246072 seconds.
	Quantizing neurons (in parallel)...
		Neuron 2 of 100 quantized successfully.
		Neuron 0 of 100 quantized successfully.
		Neuron 3 of 100 quantized successfully.
		Neuron 1 of 100 quantized successfully.
		Neuron 5 of 100 quantized successfully.
		Neuron 4 of 100 quantized successfully.
		Neuron 6 of 100 quantized successfully.
		Neuron 7 of 100 quantized successfully.
		Neuron 8 of 100 quantized successfully.
		Neuron 9 of 100 quantized successfully.
		Neuron 10 of 100 quantized successfully.
		Neuron 11 of 100 quantized successfully.
		Neuron 12 of 100 quantized successfully.
		Neuron 13 of 100 quantized successfully.
		Neuron 14 of 100 quantized successfully.
		Neuron 15 of 100 quantized successfully.
		Neuron 16 of 100 quantized successfully.
		Neuron 18 of 100 quantized successfully.
		Neuron 17 of 100 quantized successfully.
		Neuron 19 of 100 quantized successfully.
		

		Neuron 85 of 100 quantized successfully.
		Neuron 86 of 100 quantized successfully.
		Neuron 87 of 100 quantized successfully.
		Neuron 88 of 100 quantized successfully.
		Neuron 89 of 100 quantized successfully.
		Neuron 90 of 100 quantized successfully.
		Neuron 91 of 100 quantized successfully.
		Neuron 92 of 100 quantized successfully.
		Neuron 93 of 100 quantized successfully.
		Neuron 94 of 100 quantized successfully.
		Neuron 95 of 100 quantized successfully.
		Neuron 96 of 100 quantized successfully.
		Neuron 97 of 100 quantized successfully.
		Neuron 98 of 100 quantized successfully.
		Neuron 99 of 100 quantized successfully.
	done. 17.05 seconds.
Layer 1 of 3 quantized successfully in 17.37 seconds.
Quantizing layer 2 (in parallel) of 3...
	Feeding input data through hidden layers...
	done. 0.334413 seconds.
	Quantizing neurons (in parallel)...
		Neuron 1 of 2 quantized successfully.
		Neuron 0 of 2 quantized successfully.
	done. 6.33 seconds.
Layer 2 of 3 quantized success

In [36]:
trial_metrics

,q_train_size,bits,alphabet_scalar,sd_test_acc,msq_test_acc,quantization_time
plane,10391,8.0,2,0.9009,0.866335,40.640526
